## Attach Docker File

In [1]:
!cat container/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 2 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM ubuntu:16.04

RUN apt-get -y update

RUN add-apt-repository ppa:jonathonf/python-3.6
RUN apt-get update

RUN apt-get install -y build-essential python3.6 python3.6-dev python3-pip python3.6-venv
RUN apt-get install -y git

# update pip
RUN python3.6 -m pip install pip --upgrade
RUN python3.6 -m pip install wheel

RUN apt-get install -y --no-install-recommends \
         wget \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

# Install all of the packages
RUN wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py
RUN pip install numpy
RUN pip install opencv-python
RUN pip install flask
RUN pip install gevent
RUN pip install gunicorn
RUN pip install keras
RUN pip install python-chess

# Env Variables
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE
ENV PATH="/

## Building and registering the container

In [17]:
%%sh

# The name of our algorithm
algorithm_name="chessvision-algo"

cd container

# chmod +x ann/train
chmod +x src/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"
echo $fullname

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
    echo "Here"
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

echo "Yes"

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

#docker push ${fullname}

580857158266.dkr.ecr.eu-central-1.amazonaws.com/chessvision-algo:latest
Login Succeeded
Yes

Step 1/17 : FROM tiangolo/uwsgi-nginx-flask:python3.6
 ---> 7513b2ffa3ca
Step 2/17 : RUN apt-get update && apt-get install -y     python3-pip
 ---> Using cache
 ---> 2c9a7366138d
Step 3/17 : RUN pip3 install numpy
 ---> Using cache
 ---> 6dc8dcd92d79
Step 4/17 : RUN pip3 install opencv-python
 ---> Using cache
 ---> 02ae925fc9e7
Step 5/17 : RUN pip3 install gevent
 ---> Using cache
 ---> 2ded646c78ef
Step 6/17 : RUN pip3 install gunicorn
 ---> Using cache
 ---> 732359a7b4d7
Step 7/17 : RUN pip3 install keras
 ---> Using cache
 ---> 2c457abac866
Step 8/17 : RUN pip3 install python-chess
 ---> Using cache
 ---> 2389b1ae22ea
Step 9/17 : RUN pip3 install tensorflow
 ---> Using cache
 ---> d37bd89f6482
Step 10/17 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 24a365ad21cc
Step 11/17 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 36dc71523560
Step 12/17 : ENV PATH="/opt/program:${PAT

WARNING! Using --password via the CLI is insecure. Use --password-stdin.


In [10]:
# S3 prefix
prefix = 'ann-churn'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

ImportError: No module named 'sagemaker'

## Create the session

In [4]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## Upload the data for training

In [5]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

## Create an estimator and fit the model

In [6]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = f'{account}.dkr.ecr.{region}.amazonaws.com/{prefix}:latest'

classifier = sage.estimator.Estimator(
    image, 
    role, 
    1, 
    'ml.c4.2xlarge', 
    output_path="s3://{}/output".format(sess.default_bucket()),
    sagemaker_session=sess)

classifier.fit(data_location)

INFO:sagemaker:Creating training-job with name: ann-churn-2018-05-27-18-29-21-010


.............
/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
Starting the training.
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
Epoch 1/1

50833/50833 [==============================] - 3s 59us/step - loss: 0.1819 - acc: 0.9521
Epoch 1/1
50833/50833 [==============================] - 3s 59us/step - loss: 0.1722 - acc: 0.9602


Epoch 1/1

50834/50834 [==============================] - 3s 59us/step - loss: 0.2018 - acc: 0.9523
Epoch 1/1
50834/50834 [==============================] - 3s 60us/step - loss: 0.1857 - acc: 0.9607
Epoch 1/1

 3675/50834 [=>............................] - ETA: 6s - loss: 0.6567 - acc: 0.7298


50834/50834 [==============================] - 3s 61us/step - loss: 0.1933 - acc: 0.9511
Epoch 1/1

50834/50834 [==============================] - 3s 62us/step - loss: 0.2246 - acc: 0.9397
Epoch 1/1
50834/50834 [==============================] - 3s 63us/step - loss: 0.1958 - acc: 0.9571
Epoch 1/1

18875/50834 [==========>...................] - ETA: 2s - loss: 0.2875 - acc: 0.9358


50834/50834 [==============================] - 3s 63us/step - loss: 0.1825 - acc: 0.9605
Epoch 1/1

50834/50834 [==============================] - 3s 63us/step - loss: 0.1980 - acc: 0.9514
Epoch 1/1

25150/50834 [=============>................] - ETA: 1s - loss: 0.2710 - acc: 0.9220


...

50833/50833 [==============================] - 5s 98us/step - loss: 0.2318 - acc: 0.9453
Epoch 1/1
50833/50833 [==============================] - 5s 99us/step - loss: 0.2478 - acc: 0.9467
Epoch 1/1
50834/50834 [==============================] - 5s 99us/step - loss: 0.2117 - acc: 0.9550
Epoch 1/1


50834/50834 [==============================] - 5s 100us/step - loss: 0.2325 - acc: 0.9518
Epoch 1/1
50834/50834 [==============================] - 5s 101us/step - loss: 0.2197 - acc: 0.9521
Epoch 1/1
50834/50834 [==============================] - 5s 101us/step - loss: 0.2167 - acc: 0.9519


Epoch 1/1
50834/50834 [==============================] - 5s 102us/step - loss: 0.2143 - acc: 0.9554
Epoch 1/1
50834/50834 [==============================] - 5s 103us/step - loss: 0.2199 - acc: 0.9477
Epoch 1/1
25568/50834 [==============>...............] - ETA: 3s - loss: 0.3273 - acc: 0.9337


50834/50834 [==============================] - 5s 104us/step - loss: 0.2280 - acc: 0.9551
Epoch 1/1
50834/50834 [==============================] - 5s 107us/step - loss: 0.2441 - acc: 0.9486
Epoch 1/1
56482/56482 [==============================] - 7s 116us/step - loss: 0.1981 - acc: 0.9566
Training is complete.


===== Job Complete =====
Billable seconds: 560


## Deploy the model

In [14]:
from sagemaker.predictor import csv_serializer
predictor = classifier.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer)

INFO:sagemaker:Creating model with name: ann-churn-2018-05-27-18-49-07-013
INFO:sagemaker:Creating endpoint with name ann-churn-2018-05-27-18-29-21-010


-------------------------------------------------------------!

## Choose some data and use it for a prediction

In [15]:
shape=pd.read_csv("data/churn.csv", header=None)

import itertools

a = [50*i for i in range(3)]
b = [40+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]

test_data=shape.iloc[indices[:-1]]


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [16]:
# # dict_test = {col: [val] for col, val in zip(dataset.columns, X_test[1])}
# # dict_test = pd.DataFrame.from_dict(dict_test, orient = 'index')
# # s = pd.Series(dict_test, name = 'test')
# df = pd.DataFrame(X_test[1])
print(predictor.predict(test_data.values).decode('utf-8'))


0.027434496
0.37452298
0.025588198
0.026960442
0.03179186
0.04434355
0.0278371
1.0
0.11551658
1.0
0.026616087
0.032466426
0.026545195
1.0
0.02555298
0.049248382
0.032999154
1.0
1.0
0.029439388
1.0
0.9999908
0.036826607
1.0
0.026471052
0.03185896
0.037169527
1.0
1.0



## Optional cleanup

In [17]:
sess.delete_endpoint(predictor.endpoint)

INFO:sagemaker:Deleting endpoint with name: ann-churn-2018-05-27-18-29-21-010
